<a href="https://colab.research.google.com/github/ZidanMau24/UAS-AMS/blob/main/Uas_Document_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Document Similarity**





# Rekomendasi Film dengan Kemiripan Dokumen

Sistem rekomendasi adalah salah satu aplikasi pembelajaran mesin yang populer dan paling banyak diadopsi. Mereka biasanya digunakan untuk merekomendasikan entitas kepada pengguna dan entitas ini dapat berupa apa saja seperti produk, film, layanan, dan sebagainya.

Biasanya sistem pemberi rekomendasi dapat diimplementasikan dalam tiga cara:

- Simple Rule-based Recommenders / Rekomendasi Berbasis Aturan Sederhana: Biasanya berdasarkan metrik dan ambang batas global tertentu seperti popularitas film, peringkat global, dll.
- Content-based Recommenders / Rekomendasi berbasis konten: Ini didasarkan pada penyediaan entitas serupa berdasarkan entitas minat tertentu. Metadata konten dapat digunakan di sini seperti deskripsi film, genre, pemeran, sutradara, dan sebagainya
- Collaborative filtering Recommenders / Pemfilteran kolaboratif Rekomendasi: Di ​​sini kami tidak memerlukan metadata, tetapi kami mencoba memprediksi rekomendasi dan peringkat berdasarkan peringkat sebelumnya dari berbagai pengguna dan item tertentu.

Kami akan membangun sistem rekomendasi film di sini berdasarkan data\metadata yang berkaitan dengan film yang berbeda, kami mencoba dan merekomendasikan film serupa yang menarik!

Karena fokus kami bukan pada mesin rekomendasi tetapi NLP, kami akan memanfaatkan metadata berbasis teks untuk setiap film untuk mencoba dan merekomendasikan film serupa berdasarkan film tertentu yang diminati. Ini termasuk dalam rekomendasi berbasis konten.

LOAD GDRIVE

In [81]:
 from google.colab import drive
 drive.mount('/content/drive')
 import sys
 sys.path.append('/content/drive/My Drive/Colab Notebooks/UAS_AMS')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install library

In [82]:
 !pip install textsearch
 !pip install contractions
import nltk
 nltk.download('punkt')
 nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## load dan view data

In [83]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/UAS_AMS/TMDB 10000 Movies Dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   original_language  10000 non-null  object 
 2   original_title     10000 non-null  object 
 3   overview           9994 non-null   object 
 4   popularity         10000 non-null  float64
 5   release_date       10000 non-null  object 
 6   title              10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [84]:
df.head()

,id,original_language,original_title,overview,popularity,release_date,title,vote_average,vote_count
0,851644,ko,20세기 소녀,Yeon-du asks her best friend Bora to collect a...,170.540,2022-10-06,20th Century Girl,8.7,290
1,238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",86.518,1972-03-14,The Godfather,8.7,16988
2,278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,84.681,1994-09-23,The Shawshank Redemption,8.7,22748
3,240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,44.957,1974-12-20,The Godfather Part II,8.6,10293
4,667257,es,Cosas imposibles,"Matilde is a woman who, after the death of her...",32.859,2021-06-17,Impossible Things,8.6,299


In [85]:
df.info


<bound method DataFrame.info of           id original_language             original_title  \
0     851644                ko                    20세기 소녀   
1        238                en              The Godfather   
2        278                en   The Shawshank Redemption   
3        240                en      The Godfather Part II   
4     667257                es           Cosas imposibles   
...      ...               ...                        ...   
9995   11375                en         Hollywood Homicide   
9996  298614                fr  Une heure de tranquillité   
9997   50357                en                  Apollo 18   
9998   97795                en                     Emergo   
9999   11822                de                Nekromantik   

                                               overview  popularity  \
0     Yeon-du asks her best friend Bora to collect a...     170.540   
1     Spanning the years 1945 to 1955, a chronicle o...      86.518   
2     Framed in the 19

In [86]:
df = df[['title', 'overview', 'popularity']]
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2098 to 8171
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       10000 non-null  object 
 1   overview    9994 non-null   object 
 2   popularity  10000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 312.5+ KB


In [87]:
df.head()

,title,overview,popularity
2098,Black Adam,"Nearly 5,000 years after he was bestowed with ...",9137.939
466,The Woman King,"The story of the Agojie, the all-female unit o...",4835.899
1187,Black Panther: Wakanda Forever,"Queen Ramonda, Shuri, M’Baku, Okoye and the Do...",1687.653
1546,The Guardians of the Galaxy Holiday Special,On a mission to make Christmas unforgettable f...,1480.629
2077,Disenchanted,"Disillusioned with life in the city, feeling o...",1468.421


In [88]:
df.iloc[0].overview

'Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.'

In [89]:
df.dropna(axis=0, inplace= True)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9994 entries, 2098 to 8171
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       9994 non-null   object 
 1   overview    9994 non-null   object 
 2   popularity  9994 non-null   float64
dtypes: float64(1), object(2)
memory usage: 312.3+ KB


In [91]:
df.head

<bound method NDFrame.head of                                             title  \
2098                                   Black Adam   
466                                The Woman King   
1187               Black Panther: Wakanda Forever   
1546  The Guardians of the Galaxy Holiday Special   
2077                                 Disenchanted   
...                                           ...   
9671                              Serial Teachers   
3886                                       Tracks   
28                         Grave of the Fireflies   
3864                               Odds and Evens   
8171                                       G.B.F.   

                                               overview  popularity  
2098  Nearly 5,000 years after he was bestowed with ...    9137.939  
466   The story of the Agojie, the all-female unit o...    4835.899  
1187  Queen Ramonda, Shuri, M’Baku, Okoye and the Do...    1687.653  
1546  On a mission to make Christmas unforgettable f.

# Bangun Sistem Rekomendasi Film

Di sini Anda akan membangun sistem rekomendasi film Anda sendiri. Kami akan menggunakan pipa berikut:
- Teks pra-pemrosesan
- Rekayasa Fitur
- Komputasi Kemiripan Dokumen
- Temukan film serupa teratas
- Bangun fungsi rekomendasi film


## Kemiripan Dokumen / document similarity

Rekomendasi adalah tentang memahami fitur dasar yang membuat kita menyukai satu pilihan daripada yang lain. Kesamaan antar item (dalam hal ini film) adalah salah satu cara untuk memahami mengapa kita memilih satu film daripada yang lain. Ada berbagai cara untuk menghitung kesamaan antara dua item. Salah satu ukuran yang paling banyak digunakan adalah __cosine similarity__ yang telah kita gunakan di unit sebelumnya.

### Kemiripan Kosinus

Cosine Similarity digunakan untuk menghitung skor numerik untuk menunjukkan kesamaan antara dua dokumen teks. Secara matematis, ini didefinisikan sebagai berikut:

$$ cosinus(x,y) = \frac{x. y^\intercal}{||x||.||y||} $$

In [92]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['overview']))
len(norm_corpus)

9994

## Extrak TF-IDF

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(9994, 32839)

## Compute Pairwise Document Similarity

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993
0,1.000000,0.008952,0.028209,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.046738,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.008607,0.000000,0.000000
1,0.008952,1.000000,0.042893,0.0,0.029633,0.005063,0.013363,0.0,0.000000,0.0,...,0.000000,0.0000,0.014463,0.0,0.021450,0.000000,0.000000,0.007290,0.036992,0.000000
2,0.028209,0.042893,1.000000,0.0,0.011138,0.010661,0.000000,0.0,0.010486,0.0,...,0.010589,0.0067,0.000000,0.0,0.033318,0.015416,0.021654,0.015403,0.011766,0.015378
3,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.029633,0.011138,0.0,1.000000,0.008833,0.000000,0.0,0.009079,0.0,...,0.000000,0.0000,0.000000,0.0,0.010641,0.018685,0.000000,0.000000,0.010187,0.004704


## mendapatkan title / judul movies

In [95]:
movies_list = df['title'].values
movies_list, movies_list.shape

(array(['Black Adam', 'The Woman King', 'Black Panther: Wakanda Forever',
        ..., 'Grave of the Fireflies', 'Odds and Evens', 'G.B.F.'],
       dtype=object), (9994,))

## Temukan Film Serupa Teratas untuk Contoh Film

Mari ambil __Minions__ film paling populer dari kerangka data di atas dan coba temukan film paling mirip yang dapat direkomendasikan

### ambil movie ID

In [96]:
movie_idx = np.where(movies_list == 'Annabelle')[0][0]
movie_idx

964

### ambil similarities

In [97]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0., 0., 0., ..., 0., 0., 0.])

### mengambil top 10 similar movie IDs

In [98]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:11]
similar_movie_idxs

array([1925, 2094,  782, 2023,   14, 9789, 7473, 6751, 4894, 8174])

### menampilkan top 10 similar movies

In [99]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['The Jack in the Box', 'The Boy', 'Annabelle: Creation',
       'Universal Soldier', 'Jeepers Creepers: Reborn', '9', 'Inferno',
       'Wait Until Dark', 'The Night Eats the World', 'Repulsion'],
      dtype=object)

### Buat fungsi rekomendasi film untuk merekomendasikan 5 film serupa teratas untuk film apa pun


The movie title, movie title list and document similarity matrix dataframe akan diberikan sebagai input ke fungsi

In [100]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

# misal cari rekomendasi untuk film film berikut

In [101]:
popular_movies = ['Black Panther', 'Black Adam', 'Deadpool', 'Jurassic World', 'Pirates of the Caribbean: The Curse of the Black Pearl',
              'Dawn of the Planet of the Apes', 'The Hunger Games: Mockingjay - Part 1', 'Terminator Genisys', 
              'Captain America: Civil War', 'The Dark Knight', 'The Martian', 'Batman v Superman: Dawn of Justice', 
              'Pulp Fiction', 'The Godfather', 'The Shawshank Redemption', 'The Lord of the Rings: The Fellowship of the Ring',  
              'Harry Potter and the Chamber of Secrets', 'Star Wars', 'The Hobbit: The Battle of the Five Armies',
              'Iron Man']

In [102]:
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 recommended Movies:', movie_recommender(movie_title=movie, movies=movies_list, doc_sims=doc_sim_df))
    print()

Movie: Black Panther
Top 5 recommended Movies: ['Black Panther: Wakanda Forever'
 'Ultimate Avengers 2: Rise of the Panther'
 'The Tall Blond Man with One Black Shoe' 'Judas and the Black Messiah'
 "The King's Speech"]

Movie: Black Adam
Top 5 recommended Movies: ['Harry Brown' 'Dark Shadows' 'Pulse' 'The Photographer of Mauthausen'
 'Poetic Justice']

Movie: Deadpool
Top 5 recommended Movies: ['Deadpool 2' 'Day Watch' 'Mr. Brooks' 'Once Upon a Deadpool' 'Boss Level']

Movie: Jurassic World
Top 5 recommended Movies: ['Action Point' 'Jurassic Park' 'The Lost World: Jurassic Park'
 'Jurassic World Dominion' 'Futureworld']

Movie: Pirates of the Caribbean: The Curse of the Black Pearl
Top 5 recommended Movies: ["Pirates of the Caribbean: Dead Man's Chest"
 'Pirates of the Caribbean: On Stranger Tides'
 'The Pirates! In an Adventure with Scientists!'
 'Ginger Snaps Back: The Beginning' 'Red Sparrow']

Movie: Dawn of the Planet of the Apes
Top 5 recommended Movies: ['Battle for the Planet o